<a href="https://colab.research.google.com/github/Charliebond125/CE889_Group_Project/blob/main/RNN_for_group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math


import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns
import warnings

from math import sqrt

from tqdm import tqdm

np.random.seed(42)  # for reproducibility

sns.set(style="whitegrid", color_codes=True)
sns.set(font_scale=1)

pd.set_option('display.max_columns', 60)

%matplotlib inline
warnings.filterwarnings('ignore')

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

from tensorflow.python.ops.init_ops import RandomNormal
from tensorflow import keras
from keras.models import Sequential
from keras import layers
from keras.layers import SimpleRNN, Flatten, LSTM, Dropout, Dense
from keras.optimizers import SGD, Adam, RMSprop

from keras.layers import LeakyReLU

!pip install tensorflow
import os
import sys
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
from statsmodels.tsa.stattools import adfuller, acf, pacf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from IPython.display import Markdown, display, Image
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras import layers
from keras.callbacks import History
from keras import Input
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.join('..')))


from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from keras import backend as K

from numpy import save
from numpy import load

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df_train = pd.read_csv('/content/drive/MyDrive/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/test.csv')
df_store = pd.read_csv('/content/drive/MyDrive/store.csv')

df_train = df_train.merge(df_store, on=['Store'], how = 'inner')
df_test = df_test.merge(df_store, on=['Store'], how = 'inner')

In [ ]:
def conc_datetime_comp():

  '''
  Function to merge test and train data
  specifically for ease of use in processing
  and feature engineering

  '''

  df_train = pd.read_csv('/content/drive/MyDrive/train.csv')
  df_test = pd.read_csv('/content/drive/MyDrive/test.csv')
  df_store = pd.read_csv('/content/drive/MyDrive/store.csv')
  df_test['Sales'] = -1
  df_train_test = pd.concat([df_train, df_test]).reset_index(drop=True)
  df_train_test = df_train_test.merge(df_store, left_on=['Store'], right_on=['Store'],how='inner')


  # Create datetime products from the full dataframe
  
  
  df_train_test['Year'] = pd.DatetimeIndex(df_train_test['Date']).year
  df_train_test['Quarter'] = pd.DatetimeIndex(df_train_test['Date']).quarter
  df_train_test['Month'] = pd.DatetimeIndex(df_train_test['Date']).month
  df_train_test['Day'] = pd.DatetimeIndex(df_train_test['Date']).day
  df_train_test['WeekofYear'] = pd.DatetimeIndex(df_train_test['Date']).weekofyear

  df_train_test['Season'] = np.where(df_train_test["Month"].isin([3,4,5]), "Spring",
                    np.where(df_train_test["Month"].isin([6,7,8]), "Summer",
                      np.where(df_train_test["Month"].isin([9,10,11]), "Autumn",
                          np.where(df_train_test["Month"].isin([12, 1, 2]), "Winter", "None")))
                    )
  
  ''' 
  Creating more features to enrich the data
  This also means we merge the extra store data
  with the current dataset
  
  '''

  df_train_test['CompetitionOpen'] = 12 * (
            df_train_test.Year - 
                  df_train_test.CompetitionOpenSinceYear) + \
                        (df_train_test.Month - 
                            df_train_test.CompetitionOpenSinceMonth
                        )
  
  df_train_test['CompetitionOpen'] = 12 * (
          df_train_test.Year - 
                df_train_test.CompetitionOpenSinceYear) + \
                      (df_train_test.Month - 
                          df_train_test.CompetitionOpenSinceMonth
                       )

  df_train_test['PromoOpen'] = 12 * (
      df_train_test.Year - 
              df_train_test.Promo2SinceYear) + \
                  (df_train_test.WeekofYear
         -            df_train_test.Promo2SinceWeek) / 4
  
   

  df_train_test['PromoOpen'] = df_train_test.PromoOpen.apply(
      lambda x: x if x > 0 else 0)
   

  month2str = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May',
                  6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep',
            10:'Oct', 11:'Nov', 12: 'Dec'}


  df_train_test['month2str'] = df_train_test.Month.map(month2str)
  df_train_test.loc[df_train_test.PromoInterval == 0, 'PromoInterval'] = ''
  df_train_test['IsPromoMonth'] = 0
  for interval in df_train_test.PromoInterval.unique():
    interval = str(interval)
    if interval != '':
      for month in interval.split(','):
        df_train_test.loc[
            (df_train_test.month2str == month) % (df_train_test.PromoInterval == interval), 'IsPromoMonth'] = 1


  
  return df_train_test

df_train_test = conc_datetime_comp()

In [ ]:
def extract_test_data(df):
  '''
  This is possible as our test data has 0 in sales
  therefore we only select the data from our test set

  '''
  
  df_train = df.loc[df['Sales'] != -1]
  df_test = df.loc[df['Sales'] == -1]
  
  return df_train, df_test
df_train, df_test = extract_test_data(df_train_test)

In [ ]:
def clean_data(use_text_columns = True):
  
  ''' 
  Creating function which will generate our train and test data, 
  clean it, and return processed with correct values for identified features
  
  ### Note: When importing the train, test and store data ###
  ###       ensure it is pointed at the correct location  ###

  '''

  cols_num = ["Sales", "DayOfWeek", "Open", "Promo", "SchoolHoliday", "CompetitionDistance",
                "CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", "Promo2",
                "Promo2SinceWeek", "Promo2SinceYear", "Customers", "Year", "Month", "Day",
                "CompetitionOpen", "PromoOpen", "IsPromoMonth", "Store"]

  cols_text = ["StateHoliday", "StoreType", "Assortment"]
  df_train = pd.read_csv('/content/drive/MyDrive/train.csv')
  
  df_extra = pd.read_csv('/content/drive/MyDrive/store.csv')
      
  len_train_data = len(df_train)

  df_test = pd.read_csv('/content/drive/MyDrive/test.csv')

  # Setting null values of column Open in test dataset
  df_test.loc[df_test['DayOfWeek'] != 7, 'Open'] = 1
  df_test.loc[df_test['DayOfWeek'] == 7, 'Open'] = 0

  # Merge train and test dataset
  all_data = pd.concat([df_train, df_test], ignore_index=True)

  df_extra = pd.read_csv('/content/drive/MyDrive/store.csv')
  df_full = pd.concat([df_train, df_test]).reset_index(drop=True)

  # Merge extra information about stores
  all_data = df_full.merge(df_extra, left_on=['Store'], right_on=['Store'], how='left')

  # Separate date in Year, Month and Day
  all_data.loc[all_data['StateHoliday'] == 0, 'StateHoliday'] = 'd'
  all_data['Year'] = pd.DatetimeIndex(all_data['Date']).year
  all_data['Month'] = pd.DatetimeIndex(all_data['Date']).month
  all_data['Day'] = pd.DatetimeIndex(all_data['Date']).day
  all_data['WeekOfYear'] = pd.DatetimeIndex(all_data['Date']).weekofyear

  # Calculate competition open in months
  all_data['CompetitionOpen'] = 12 * (
      all_data.Year - all_data.CompetitionOpenSinceYear) + \
          (all_data.Month - all_data.CompetitionOpenSinceMonth)

  # Calculate promo open time in months
  all_data['PromoOpen'] = 12 * (
      all_data.Year - all_data.Promo2SinceYear) + \
          (all_data.WeekOfYear - all_data.Promo2SinceWeek) / 4.0
  
  all_data['PromoOpen'] = all_data.PromoOpen.apply(
      lambda x: x if x > 0 else 0)
  
  all_data.loc[all_data.Promo2SinceYear == 0, 'PromoOpen'] = 0
  
  # Transform month interval in a boolean column 
  month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
                7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
  all_data['monthStr'] = all_data.Month.map(month2str)
  all_data.loc[all_data.PromoInterval == 0, 'PromoInterval'] = ''
  all_data['IsPromoMonth'] = 0
  for interval in all_data.PromoInterval.unique():
      interval = str(interval)
      if interval != '':
          for month in interval.split(','):
              all_data.loc[(all_data.monthStr == month) & (all_data.PromoInterval == interval), 'IsPromoMonth'] = 1

  data_numeric = all_data[cols_num]
  
  # Fill NAN values
  # Only column CompetitionDistance is fill NaN with a median value
  data_numeric['CompetitionDistance'].fillna(data_numeric['CompetitionDistance'].median(), inplace = True)

  # Other values is fill with zero
  data_numeric.fillna(0, inplace = True)

  if (use_text_columns):
      data_text = all_data[cols_text]
      data_text = pd.get_dummies(data_text, dummy_na=False)

      complete_data = pd.concat([data_numeric, data_text], axis = 1)

      df_train = complete_data.iloc[:len_train_data,:]
      df_test = complete_data.iloc[len_train_data:,:]
  else:
      df_train = data_numeric.iloc[:len_train_data,:]
      df_test = data_numeric.iloc[len_train_data:,:]

  return df_train, df_test

In [ ]:
# Insepcting closed stores and removing if any

'''
As can be seen in the below table, we have stores which were closed when it was not a state holiday

These will be removed as they may give us outliers and affect the data

'''
df_train_closed = df_train[(df_train.Open ==0) & (df_train.Sales == 0)
                  & (df_train.StateHoliday_0 ==0 )
                  & (df_train.SchoolHoliday ==0 )].head()

df_train_closed.head()

In [ ]:
df_train = df_train.drop(df_train[
    (df_train.Open ==0) 
    & (df_train.Sales == 0) & (df_train.SchoolHoliday ==0 )
    & (df_train.StateHoliday_0 == 0) & (df_train.StateHoliday_a == 0) 
    & (df_train.StateHoliday_b==0) & (df_train.StateHoliday_c==0) 
    & (df_train.StateHoliday_d==0)].index)

In [ ]:
df_train = df_train.drop(['StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c', 'StateHoliday_c', 'StateHoliday_d'], axis=1)

In [ ]:
df_test = df_test.drop(['StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c', 'StateHoliday_c', 'StateHoliday_d'], axis=1)

In [ ]:
def scaler_function_train(scaler_X, scaler_y):
  from sklearn.preprocessing import RobustScaler, StandardScaler
  robust = RobustScaler()
  scaler = StandardScaler()

  X_train = df_train.drop(['Sales'], axis=1); ''' feature variable '''
  y_train = np.array(df_train["Sales"]).reshape((len(X_train), 1)); ''' y target '''
  X_train = robust.fit_transform(X_train)
  y_train = robust.fit_transform(y_train)
  X_train = scaler.fit_transform(X_train)
  y_train = scaler.fit_transform(y_train)

  X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

  return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = scaler_function_train(df_train, df_test)

In [ ]:
from numpy import load

X_train = load('/content/drive/MyDrive/Scaled_data/X_train.npy')
X_test = load('/content/drive/MyDrive/Scaled_data/X_test.npy')
y_train = load('/content/drive/MyDrive/Scaled_data/y_train.npy')
y_test = load('/content/drive/MyDrive/Scaled_data/y_test.npy')

In [ ]:
def rmspe_val(y_true, y_pred):
    '''
    RMSPE calculus to validate evaluation metric about the model
    '''
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true), axis=0))[0]

In [ ]:
def rmspe(y_true, y_pred):
    '''
    RMSPE calculus to use during training phase
    '''
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true), axis=-1))

In [ ]:
def rmse(y_true, y_pred):
    '''
    RMSE calculus to use during training phase
    '''
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
def create_timestep(input_x, window_size):
  
      nrows = input_x.shape[0] - window_size + 1
      p,q = input_x.shape
      m,n = input_x.strides
      strided = np.lib.stride_tricks.as_strided
      output_X = strided(input_x,shape=(nrows,window_size,q),strides=(m,m,n))
  
      return output_X

In [ ]:
''' 
Reshaping an array to give it (n, n, n)
is necessary before feeding into a RNN

window_size --> passes in the current iteration, as well as n previous layers.

window_size is used for noisy or erroneous data, the window is used to mitigate

window_size can be defined from (1-->n)

'''

# create_timestep(input, window_size)

In [ ]:
def RNN_model():
  ''' 
  RNN model for group project 
  Timestep  = Processing of the input through the recurrent unit

  Input Layer --> Hidden/Recurrent Layer --> Output Layer

  Each section of the hidden layer contains a recurrent unit, which processes information
  for n timesteps. Passes through a hidden state and an input for each timestep using an
  activation function.

  Each Recurrent unit contains three parameters, which must be specified before feeding 
  into the model. RNN uses folding in time to train the weights of the network. This is called
  Back Propogation through Time i.e. BPTT.
  
  '''

  #813767, 1, 26
  initializer = RandomNormal(mean=0.0, stddev=0.05, seed=None)

  model= Sequential(name="First-RNN-Model") # Model

  model.add(Input(shape=(4, 26), name= "Input-Layer")) # Input layer

  ''' First hidden layer ''' 
  model.add(layers.SimpleRNN(units=256, input_dim=X_train.shape[1], 
                             activation="tanh", return_sequences=True,
                             kernel_initializer=initializer,
                             name='First-Hidden-RNN'))
  model.add(Dropout(0.2))

  ''' Second hidden layer '''
  model.add(layers.SimpleRNN(units=512, input_dim=X_train.shape[1], 
                              activation="tanh", return_sequences=True,
                                kernel_initializer=initializer,
                                  name='Second-Hidden_RNN'))
  model.add(Dropout(0.2))

  ''' third hidden layer '''
  model.add(layers.SimpleRNN(units=512, activation="relu",
                               return_sequences=True,
                                 kernel_initializer=initializer,
                                   name='Third-Hidden-RNN'))
  model.add(Dropout(0.2))

  ''' fourth hidden layer '''
  model.add(layers.SimpleRNN(units=256, activation="relu",
                               return_sequences=True,
                                 kernel_initializer=initializer, 
                                   name='Fourth-Recurrent-Layer-3'))
  model.add(Dropout(0.2))

  ''' fifth hidden layer '''
  model.add(layers.Dense(units=16, activation="tanh"))

  ''' output layer '''
  model.add(layers.Dense(units=1, activation='linear'))
  
  adam = Adam(lr=1e-7)

  model.compile(optimizer=adam,
                loss="mean_squared_error",
                metrics=[rmse, rmspe])
  
  return model

rnn_model = RNN_model()
rnn_model.summary()

In [ ]:
print("Building model...")

filepath = 'RNN_model_weights.hdf5'

checkpoint = ModelCheckpoint(filepath, 
                                    monitor='val_loss',
                                      verbose=0,
                                          save_best_only=True,
                                             mode='max'
                                                        )

earlystopping = EarlyStopping(monitor='val_loss',
                                    patience = 10,
                                        verbose=0,
                                              mode='max',
                                                baseline=0.6
                                                        )


In [ ]:
history = model.fit(X_train, 
                        y_train, 
                          validation_split=0.2,
                            batch_size=batch_size, 
                              epochs = nb_epoch,
                    callbacks=earlystopping, shuffle=True
                          )